<a href="https://colab.research.google.com/github/PedroPavanii/projeto-shark/blob/main/Shark_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **aumento de casos portos brasil. A cada celula discorro a pergunta chave e como cheguei no resultado com códigos comentados. recife se comportou de forma diferentelocalidade/data aumento depois do porto maior que as outras localizações**

# **Limpando os dados**

In [1]:
import pandas as pd
import numpy as np
import re
from google.colab import drive 
drive.mount('/drive')
tabela_shark = pd.read_excel('/drive/MyDrive/Shark/GSAF5.xls')


Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


## **Explorando o dado bruto:**

In [2]:
tabela_shark.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6865 entries, 0 to 6864
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    6840 non-null   object 
 1   Year                    6838 non-null   float64
 2   Type                    6820 non-null   object 
 3   Country                 6790 non-null   object 
 4   Area                    6362 non-null   object 
 5   Location                6279 non-null   object 
 6   Activity                6257 non-null   object 
 7   Name                    6621 non-null   object 
 8   Sex                     6263 non-null   object 
 9   Age                     3869 non-null   object 
 10  Injury                  6805 non-null   object 
 11  Fatal (Y/N)             6282 non-null   object 
 12  Time                    3339 non-null   object 
 13  Species                 3757 non-null   object 
 14  Investigator or Source  6821 non-null   

In [3]:
tabela_shark.head(5)

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,...,Species,Investigator or Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,25-Mar-2023,2023.0,Unprovoked,USA,Florida,"Blue Fish Channel, Key West, Monroe County",Kite-Surfing,Kevin Carlton Scott,M,67,...,NaN,"Yahoo News, 3/28/2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,22-Mar-2023,2023.0,Unprovoked,AUSTRALIA,Western Australia,Delta Island,Wading,Adam Norton,M,30s,...,NaN,"A. Currie, GSAF",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,19-Mar-2023,2023.0,Unprovoked,USA,Hawaii,Anaehoomalu Bay,Swimming,male,M,60,...,NaN,"Khon2, 3/19/2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17-Mar-2023,2023.0,Unprovoked,MEXICO,Quintana Roo,Cancún,Swimming,Dillon Armijo,M,10,...,NaN,"Denver7, 3/29/2023",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16-Mar-2023,2023.0,Unprovoked,AUSTRALIA,Western Australia,Cheynes Beach,Surf-sking,female,F,NaN,...,5m to 6m shark,"B. Myatt, GSAF",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
tabela_shark.tail(5)

,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,...,Species,Investigator or Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
6860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN
6861,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN
6862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN
6863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN
6864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,http://sharkattackfile.net/spreadsheets/pdf_di...,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
tabela_shark.isna().sum()

Date                        25
Year                        27
Type                        45
Country                     75
Area                       503
Location                   586
Activity                   608
Name                       244
Sex                        602
Age                       2996
Injury                      60
Fatal (Y/N)                583
Time                      3526
Species                   3108
Investigator or Source      44
pdf                         66
href formula                46
href                        69
Case Number                 67
Case Number.1               68
original order              66
Unnamed: 21               6864
Unnamed: 22               6863
dtype: int64

## **Funções utilizadas no processo:**

In [6]:
#LISTAS DE FUNÇÕES UTILIZADAS NO CODIGO
def limpar_coluna_string(df, coluna):
    """
    Função que remove espaços antes e depois das palavras em uma coluna de um DataFrame,
    converte para caixa alta e remove caracteres do tipo ? e / do meio das palavras.
    
    Argumentos:
    - df: pandas.DataFrame - O DataFrame que contém a coluna a ser limpa.
    - coluna: str - O nome da coluna a ser limpa.
    
    Retorno:
    - dataframe
    """
    import re

    # verifica se a coluna é uma instância de pandas.Series
    if isinstance(df[coluna], pd.Series):
        # converte a coluna em uma string
        df[coluna] = df[coluna].astype(str)

    # remove espaços antes e depois das palavras e converte para caixa alta
    df[coluna] = df[coluna].str.strip().str.upper()

    # remove caracteres ? e / do meio das palavras
    df[coluna] = df[coluna].apply(lambda x: re.sub('[/?\.]', '', x))
    return df

## **Realizando a limpeza, subtituições e retiradas do DataFrame original:**

In [7]:
#RETIRANDO LINHAS 100% NULAS E COLUNAS QUE NÃO TEM SIGNIFICADO PARA O ESTUDO E COLOCANDO 'UNKNOWN' EM COLUNAS QUE SÃO RELEVANTES.
tabela_shark = tabela_shark.drop([ 'Investigator or Source', 'pdf', 'href formula', 'href', 'Case Number','Case Number.1', 'original order', 'Unnamed: 21', 'Unnamed: 22'], axis=1).dropna(how = 'all')
tabela_hark = tabela_shark['Location'].fillna('UNKNOWN', inplace = True)
tabela_hark = tabela_shark['Area'].fillna('UNKNOWN', inplace = True)

In [8]:
#TRATAMENTO DE STRING PARA CAIXA ALTA,RETIRANDO ESPAÇOS NO COMEÇO E FINAL. FUNÇÃO DESCRITA NO TOPICO DE FUNÇÕES. MELHORANDO A VISUALIZAÇÃO DA COLUNA DE PAÍSES. 
tabela_shark = limpar_coluna_string(tabela_shark,'Country')
#tabela_shark = tabela_shark[tabela_shark['Country'] == 'BRAZIL']

In [9]:
# TRATAMENTO DA COLUNA AREA, SUBSTITUICAO DOS NUMEROS DE COORDENADAS POR OUTROS TENDO EM VISTA AS COORDENADAS ESTAREM INCOMPLETAS.
pattern = r'\d+[º.,]\s*[SNLW]'
tabela_shark['Area'] = tabela_shark['Area'].replace('.*\d.*', 'UNKNOWN', regex = True )


In [10]:
# TRATAMENTO DA COLUNA AREA, SUBSTITUICAO DOS NUMEROS DE COORDENADAS POR OUTROS TENDO EM VISTA AS COORDENADAS ESTAREM INCOMPLETAS E REMOVENDO.
tabela_shark['Location'] = tabela_shark['Location'].replace('.*\d.*', 'OUTROS', regex = True )
tabela_shark['Location'] = tabela_shark['Location'].replace('[.?]', ' ', regex = True)

In [14]:
#REALIZO A CONVERSAO DA COLUNA DATE EM STR E PASSO PORA O FORMATO DATETIME
tabela_shark['Date'] = pd.to_datetime(tabela_shark['Date'], dayfirst=True, errors='coerce')
#CRIO UMA NOVA COLUNA COM OS ANOS CONTIDOS NA COLUNA DATA
tabela_shark['Years'] = tabela_shark['Date'].apply(lambda x: x.year if x is not pd.NaT else pd.NaT)
# SUBSTITUO OS NAN POR UNKNOWN
tabela_shark['Date'] = tabela_shark['Date'].fillna('unknown')
tabela_shark['Years'] = tabela_shark['Date'].fillna('unknown')

#REALIZO A MELHORA ESTETICA DAS COLUNAS
tabela_shark['Date'] = tabela_shark['Date'].astype(str).str.replace('00:00:00', ' ')
tabela_shark['Year'] = tabela_shark['Year'].astype(str).str.replace(r'(?<![A-Za-z0-9])0\.0','unknown', regex = True)
tabela_shark['Year'] = tabela_shark['Year'].replace('1000.0','unknown')
tabela_shark['Year'] = tabela_shark['Year'].replace('77.0','unknown')
tabela_shark['Year'] = tabela_shark['Year'].replace('5.0','unknown')
tabela_shark['Year'] = tabela_shark['Year'].replace('nan','unknown')

In [15]:
tabela_shark['Year'].unique()

array(['2023.0', '2022.0', '2021.0', '2020.0', '2019.0', '2018.0',
       '2017.0', 'unknown', '2016.0', '2015.0', '2014.0', '2013.0',
       '2012.0', '2011.0', '2010.0', '2009.0', '2008.0', '2007.0',
       '2006.0', '2005.0', '2004.0', '2003.0', '2002.0', '2001.0',
       '2000.0', '1999.0', '1998.0', '1997.0', '1996.0', '1995.0',
       '1984.0', '1994.0', '1993.0', '1992.0', '1991.0', '1990.0',
       '1989.0', '1969.0', '1988.0', '1987.0', '1986.0', '1985.0',
       '1983.0', '1982.0', '1981.0', '1980.0', '1979.0', '1978.0',
       '1977.0', '1976.0', '1975.0', '1974.0', '1973.0', '1972.0',
       '1971.0', '1970.0', '1968.0', '1967.0', '1966.0', '1965.0',
       '1964.0', '1963.0', '1962.0', '1961.0', '1960.0', '1959.0',
       '1958.0', '1957.0', '1956.0', '1955.0', '1954.0', '1953.0',
       '1952.0', '1951.0', '1950.0', '1949.0', '1948.0', '1848.0',
       '1947.0', '1946.0', '1945.0', '1944.0', '1943.0', '1942.0',
       '1941.0', '1940.0', '1939.0', '1938.0', '1937.0', '193

In [13]:
#tabela_shark = tabela_shark[tabela_shark['Country'] == 'BRAZIL']